In [23]:
import arcpy
from arcpy import env, mp
import pandas as pd
import requests
import os
import zipfile
import numpy as np

landcover_url = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
out_dir = r'C:\Users\benla\Desktop\Grad_School\Classes\GIS5572_SpatialDataScience\Assignments\Group_Project\QAQC\QAQC_arc'
  
output_crs = arcpy.SpatialReference(26915)
output_landcover = os.path.join(out_dir, 'reprojected_nlcd.tif')

In [24]:
def download_landcover(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded: {save_path}")
    else:
        print(f"Failed to download: {url}")
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.join(out_dir, 'landcover'))

def crs_qa_step(raster, output_raster):
    desc = arcpy.Describe(raster)
    current_crs = desc.spatialReference

    # If no coordinate system is defined, set it first
    if current_crs is None or current_crs.name == "Unknown":
        print(f"Defining projection for {raster}")
        arcpy.DefineProjection_management(raster, output_crs)
        current_crs = output_crs  # Update current_crs after defining

    # Check if reprojection is necessary
    if current_crs.factoryCode != output_crs.factoryCode:
        print(f"Reprojecting {raster} from {current_crs.name} to {output_crs.name}")
        arcpy.ProjectRaster_management(raster, output_raster, output_crs)
    else:
        print(f"{raster} is already in the desired CRS ({output_crs.name})")

In [27]:
# Download the .tif file
download_landcover(landcover_url, os.path.join(out_dir, 'landcover.zip'))
landcover_path = os.path.join(out_dir, r'landcover', r'NLCD_2019_Land_Cover.tif')
landcover = arcpy.Raster(landcover_path)     

Downloaded: C:\Users\benla\Desktop\Grad_School\Classes\GIS5572_SpatialDataScience\Assignments\Group_Project\QAQC\QAQC_arc\landcover.zip


In [28]:
# Run CRS QA
crs_qa_step(landcover_path, output_landcover)

C:\Users\benla\Desktop\Grad_School\Classes\GIS5572_SpatialDataScience\Assignments\Group_Project\QAQC\QAQC_arc\landcover\NLCD_2019_Land_Cover.tif is already in the desired CRS (NAD_1983_UTM_Zone_15N)


In [44]:
# Define output paths
mn_boundary = os.path.join(out_dir, "mn_boundary.shp")
mn_boundary_projected = os.path.join(out_dir, "mn_boundary_utm15.shp")
corrected_landcover = os.path.join(out_dir, 'nlcd_mn_utm15.tif')
clipped_raster = os.path.join(out_dir, "nlcd_clipped.tif")

# Reference the "USA States Generalized" layer in the active map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_doc = aprx.listMaps()[0]  # Assuming it's the first map
states_layer = map_doc.listLayers("USA States Generalized")[0]

# Extract only Minnesota from the USA States Generalized layer
arcpy.analysis.Select(
    in_features=states_layer,
    out_feature_class=mn_boundary,
    where_clause="STATE_NAME = 'Minnesota'"
)

<Result 'C:\\Users\\benla\\Desktop\\Grad_School\\Classes\\GIS5572_SpatialDataScience\\Assignments\\Group_Project\\QAQC\\QAQC_arc\\nlcd_clipped.tif'>

In [50]:
# Reproject NLCD land cover data to NAD83 UTM Zone 15N before clipping
arcpy.ProjectRaster_management(
    in_raster=landcover_path, 
    out_raster=corrected_landcover, 
    out_coor_system=arcpy.SpatialReference(26915),  # EPSG: 26915 (NAD83 UTM Zone 15N)
    resampling_type="NEAREST"
)

# Reproject the boundary shapefile to match the raster's CRS (EPSG:26915)
arcpy.Project_management(
    in_dataset=mn_boundary, 
    out_dataset=mn_boundary_projected, 
    out_coor_system=arcpy.SpatialReference(26915)  # EPSG: 26915 (NAD83 UTM Zone 15N)
)

<Result 'C:\\Users\\benla\\Desktop\\Grad_School\\Classes\\GIS5572_SpatialDataScience\\Assignments\\Group_Project\\QAQC\\QAQC_arc\\mn_boundary_utm15.shp'>

In [51]:
# Clip with the reprojected boundary
arcpy.Clip_management(
    in_raster=corrected_landcover, 
    out_raster=clipped_raster,
    in_template_dataset=mn_boundary_projected,
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\benla\\Desktop\\Grad_School\\Classes\\GIS5572_SpatialDataScience\\Assignments\\Group_Project\\QAQC\\QAQC_arc\\nlcd_clipped.tif'>

In [55]:
# Function to QAQC the NLCD extent using the Minnesota boundary
def validate_nlcd(nlcd_raster, boundary_shapefile):
    if not arcpy.Exists(nlcd_raster):
        print(f"Error: Raster {nlcd_raster} does not exist.")
        return
    
    # Describe the NLCD raster
    desc_raster = arcpy.Describe(nlcd_raster)
    nlcd_extent = desc_raster.extent
    nlcd_crs = desc_raster.spatialReference
    
    # Describe the boundary
    desc_boundary = arcpy.Describe(boundary_shapefile)
    boundary_extent = desc_boundary.extent
    boundary_crs = desc_boundary.spatialReference
    
    # Ensure CRS is defined
    if nlcd_crs is None or nlcd_crs.name == "Unknown":
        print(f"Error: The NLCD raster {nlcd_raster} has an undefined coordinate system.")
        return
        
    # Check if the raster and boundary are in the same coordinate system
    if nlcd_crs.factoryCode != boundary_crs.factoryCode:
        print(f"Warning: Coordinate system mismatch! NLCD is in {nlcd_crs.name} (EPSG:{nlcd_crs.factoryCode}), but boundary is in {boundary_crs.name} (EPSG:{boundary_crs.factoryCode}).")
        print("Using the projected boundary for comparison (mn_boundary_projected)...")
        
        # Make sure we're using the projected boundary for the comparison
        if "utm" not in boundary_shapefile.lower() and nlcd_crs.factoryCode == 26915:
            print("Error: You need to use the projected boundary (mn_boundary_utm15.shp) for this comparison.")
            return
    
    # Check if the raster extent is within the Minnesota boundary extent
    # Add a small buffer to account for precision differences
    buffer = 0.001  # Small buffer in coordinate units
    within_extent = (
        nlcd_extent.XMin >= (boundary_extent.XMin - buffer)
        and nlcd_extent.YMin >= (boundary_extent.YMin - buffer)
        and nlcd_extent.XMax <= (boundary_extent.XMax + buffer)
        and nlcd_extent.YMax <= (boundary_extent.YMax + buffer)
    )
    
    if within_extent:
        print(f"{nlcd_raster} is within the expected Minnesota boundary.")
    else:
        print(f"Warning: {nlcd_raster} might be outside the expected Minnesota boundary!")
        print(f"Boundary extent: XMin={boundary_extent.XMin}, YMin={boundary_extent.YMin}, XMax={boundary_extent.XMax}, YMax={boundary_extent.YMax}")
        print(f"NLCD extent: XMin={nlcd_extent.XMin}, YMin={nlcd_extent.YMin}, XMax={nlcd_extent.XMax}, YMax={nlcd_extent.YMax}")

# Function to check NLCD completeness and valid land cover values
def check_nlcd_completeness(nlcd_raster):
    if not arcpy.Exists(nlcd_raster):
        print(f"Error: Raster {nlcd_raster} does not exist.")
        return

    # Create a raster object
    raster = arcpy.Raster(nlcd_raster)

    # Check if the raster has a VAT (Value Attribute Table)
    has_attribute_table = arcpy.management.BuildRasterAttributeTable(nlcd_raster, "Overwrite")[0]
    
    if has_attribute_table:
        print("NLCD raster has a valid attribute table.")

        # Check for missing or unexpected land cover values
        print("Checking land cover values in NLCD dataset...")
        valid_landcover_codes = {11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 52, 71, 81, 82, 90, 95}  # Standard NLCD codes
        
        with arcpy.da.SearchCursor(nlcd_raster, ["VALUE", "COUNT"]) as cursor:
            for row in cursor:
                landcover_class, count = row[0], row[1]
                if landcover_class not in valid_landcover_codes:
                    print(f"Warning: Unexpected land cover class {landcover_class} found with {count} cells.")

        print("Land cover validation complete.")
    else:
        print("No attribute table found. Cannot check land cover values.")

    # Calculate the total number of classified cells in the NLCD raster
    total_cells = raster.width * raster.height
    print(f"Total classified cells in NLCD: {total_cells}")


In [56]:
# Run QAQC validation on the final clipped raster
validate_nlcd(clipped_raster, mn_boundary_projected)
check_nlcd_completeness(clipped_raster)

Boundary extent: XMin=190687.90811424778, YMin=4816425.970984291, XMax=758772.0579467257, YMax=5471006.466779861
NLCD extent: XMin=190670.0, YMin=4816420.0, XMax=758780.0, YMax=5471020.0
NLCD raster has a valid attribute table.
Checking land cover values in NLCD dataset...
Land cover validation complete.
Total classified cells in NLCD: 413205340
